In [3]:
from torch.utils.data import DataLoader
from dataset import LichessStream, collate
from pathlib import Path
import torch
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 75000
num_epochs = 8

learning_rate = 0.0001

# ----- Dataset ----------------------------------

# Get the batches of preprocessed data
batches = list(Path("data/split_data").glob("batch_*"))

In [12]:
import json

total = 0
for batch in batches[:4]:
    with open(batch / "next_moves.json") as f:
        data = json.load(f)
        total += len(data)

total/75000

35.847053333333335

In [ ]:
from torch.utils.data import DataLoader
from dataset import LichessStream, collate
from pathlib import Path
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 75000
num_epochs = 8

learning_rate = 0.0001

# ----- Dataset ----------------------------------

# Get the batches of preprocessed data
batches = list(Path("data/split_data").glob("batch_*"))

# Create train/test split
split = int(0.8 * len(batches))

# train_stream = LichessStream(batches=batches[:split])
# valid_stream = LichessStream(batches=batches[split:])

train_stream = LichessStream(batches=batches[:1])
valid_stream = LichessStream(batches=batches[1:2])

train_loader = DataLoader(
    train_stream,
    batch_size=batch_size,
    num_workers=3,
    collate_fn=collate,
    prefetch_factor=4,
    # pin_memory=True,
    drop_last=True,
)

valid_loader = DataLoader(
    valid_stream,
    batch_size=batch_size,
    # num_workers=1,
    collate_fn=collate,
    # prefetch_factor=4,
    pin_memory=True,
    drop_last=True,
)


# Need to come up with a unified "vocab"
idx_to_move = train_stream.idx_to_move


# ----- Model -----------------------------------
class ImitationModel(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(ImitationModel, self).__init__()

        self.seq = torch.nn.Sequential(
            torch.nn.Linear(in_features, 20_000),
            torch.nn.ReLU(),
            torch.nn.Linear(20_000, 10_000),
            torch.nn.ReLU(),
            torch.nn.Linear(10_000, 5_000),
            torch.nn.ReLU(),
            torch.nn.Linear(5_000, out_features)
        )

    def forward(self, x):
        x = self.seq(x)
        return x


model = ImitationModel(in_features=837, out_features=len(idx_to_move)).to(device)


# ----- Training -----------------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_fn = torch.nn.CrossEntropyLoss()

scaler = torch.amp.GradScaler()

for epoch in range(num_epochs):

    # ---- TRAIN ----
    model.train()
    train_loss = train_correct = train_seen = 0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device, dtype=torch.float32), batch_y.to(device).long()
        break

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7338e8dab6a0>
Traceback (most recent call last):
  File "/home/msaad/miniconda3/envs/chess/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1663, in __del__
    self._shutdown_workers()
  File "/home/msaad/miniconda3/envs/chess/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1627, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/msaad/miniconda3/envs/chess/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/msaad/miniconda3/envs/chess/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/msaad/miniconda3/envs/chess/lib/python3.12/multiprocessing/connection.py", line 1118, in wait
    ready = selector.select(timeout)
            ^^^^^^^^

In [13]:
from torch.utils.data import DataLoader
from dataset import LichessStream, collate
from pathlib import Path
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 75000
num_epochs = 8

learning_rate = 0.0001

# ----- Dataset ----------------------------------

# Get the batches of preprocessed data
batches = list(Path("data/split_data").glob("batch_*"))

# Create train/test split
split = int(0.8 * len(batches))

# train_stream = LichessStream(batches=batches[:split])
# valid_stream = LichessStream(batches=batches[split:])

train_stream = LichessStream(batches=batches[:1])
valid_stream = LichessStream(batches=batches[1:2])

train_loader = DataLoader(
    train_stream,
    batch_size=batch_size,
    # num_workers=3,
    collate_fn=collate,
    # prefetch_factor=4,
    # pin_memory=True,
    drop_last=True,
)


In [2]:

train_seen = 0
count = 0
for batch_x, batch_y in train_loader:
    train_seen += batch_y.size(0)
    count += 1

    if count % 10 == 0:
        print(train_seen)

KeyboardInterrupt: 

In [14]:
gen =iter(train_loader)

In [15]:
d = next(gen)

In [47]:
%%timeit
next(gen)

160 ms ± 6.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
train_seen

75000